In [4]:
from dotenv import load_dotenv
import openai 
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_API_BASE = os.getenv("OPENAI_API_BASE")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
OPENAI_DEPLOYMENT_VERSION = os.getenv("OPENAI_DEPLOYMENT_VERSION")

OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME = os.getenv("OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME")
OPENAI_ADA_EMBEDDING_MODEL_NAME = os.getenv("OPENAI_ADA_EMBEDDING_MODEL_NAME")

#init Azure OpenAI 
openai.api_type = "azure"
openai.api_version = OPENAI_DEPLOYMENT_VERSION
openai.api_base = OPENAI_API_BASE
openai.api_key = OPENAI_API_KEY

In [17]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import TokenTextSplitter
from langchain.vectorstores import FAISS

embeddings=OpenAIEmbeddings(deployment=OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME,
                            model=OPENAI_ADA_EMBEDDING_MODEL_NAME,
                            openai_api_base=OPENAI_API_BASE,
                            openai_api_type="azure",
                            chunk_size=1)

loader = DirectoryLoader("./data/stories/", glob="*.txt", loader_cls=TextLoader)

documents = loader.load()
text_splitter = TokenTextSplitter(chunk_size=10000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

db = FAISS.from_documents(documents=docs, embedding=embeddings)

db.save_local("./dbs/stories/faiss_index")


In [18]:
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.vectorstores import FAISS 
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate

def ask_question(qa, question): 
    result = qa({"query": question})
    print("Question:", question)
    print("Answer:", result["result"])

def ask_question_with_context(qa, question, chat_history): 
    query = "what is this story?"
    result = qa({"question": question, "chat_history": chat_history})
    print("answer:", result["answer"])
    chat_history = [(query, result["answer"])]
    return chat_history

llm = AzureChatOpenAI(deployment_name=OPENAI_DEPLOYMENT_NAME,
                      model_name=OPENAI_MODEL_NAME,
                      openai_api_base=OPENAI_API_BASE,
                      openai_api_version=OPENAI_DEPLOYMENT_VERSION,
                      openai_api_key=OPENAI_API_KEY,
                      openai_api_type="azure")
    
embeddings=OpenAIEmbeddings(deployment=OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME,
                                model=OPENAI_ADA_EMBEDDING_MODEL_NAME,
                                openai_api_base=OPENAI_API_BASE,
                                openai_api_type="azure",
                                chunk_size=1)     

vectorStore = FAISS.load_local("./dbs/stories/faiss_index", embeddings)                          

retriever = vectorStore.as_retriever(search_type="similarity", search_kwargs={"k": 2})

QUESTION_PROMPT = PromptTemplate.from_template("""Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.
    Chat History:
    {chat_history}
    Follow Up Input: {question}
    Standalone question:""")

qa = ConversationalRetrievalChain.from_llm(llm=llm,
                                            retriever=retriever,
                                            condense_question_prompt=QUESTION_PROMPT,
                                            return_source_documents=True,
                                            verbose=False)

In [19]:
chat_history = []
while True:
    query = input('you: ')
    if query == 'q': 
        break
    chat_history = ask_question_with_context(qa, query, chat_history)

answer: 这个故事涉及到多个人物，没有一个明确的主角。其中包括雄一、菊池文彦、唐泽雪穗、川岛江利子、藤村都子等人。
answer: The story involves multiple characters, including a woman named Meika, her father, her stepmother Tangze Xuexi, her step-uncle Xiaozhong, her step-cousin Youda, a man named Sasagaki, and a man named Shuichi. Additionally, there are two other characters named Kikuchi and Matsupura who appear briefly.
answer: Based on the context provided, it is difficult to determine a clear answer to this question. However, there are some possible characterizations mentioned by the characters in the text, such as:

- Tangze Xuexi is a young and successful female entrepreneur who has opened several businesses.
- Some people envy her success and may spread rumors about her, saying that she is money-oriented and willing to use others to make a profit.
- She is also known for her beauty and has a male fan club supporting her.
- Tangze Xuexi is a talented and smart student, as she was able to open her own businesses at a young age and has 

ValueError: Azure has not provided the response due to a content filter being triggered